# LAB31: Prompt Orchestration



## Configuración de la API de OpenAI
Este código importa las librerías necesarias para trabajar con la API de OpenAI, carga las variables de entorno desde un archivo .env y configura la clave de la API de OpenAI para ser utilizada en las peticiones a la API.


In [1]:
!pip install langchain openai langchain_experimental langchain_openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.59
    Uninstalling langchain-core-0.3.59:
      Successfully uninstalled langchain-core-0.3.59


In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


## Definimos los modelos que usaremos

En este ejemplo usaremos como SLM (Small Language Model) a GPT-3.5 y como LLM a GPT-4o


In [5]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

slm = ChatOpenAI(model_name="gpt-4.1-mini", temperature = 0)
llm = ChatOpenAI(model_name="gpt-4.1", temperature = 0.7)

## Definimos el Prompt Orchestrator

Este prompt será el encargado de clasificar la solicitud del usuario en alguna de las categorías disponibles. Debemos lograr que la respuesta siempre sea una cadena únicamente con la categoría, sin explicaciones ni nada.


In [3]:
prompt_orchestrator = """
You are an expert at categorizing intentionality in chatbot messages.
You will receive a request from the user and you will have to classify it into one of the following categories:

- order_status: In this category we will only have user requests that have to do with the status of the user's orders
- product_information: In this category we will only have user requests that have to do with questions regarding the company's products
- order_cancellation: In this category we will only enter requests to cancel purchase orders
- support: In this category we will only have more general help requests about the online store.

If you don't find any category just return support.

If the user request is related to something away from an ecommerce return NULL

We only want the category, we don't want you to explain to us how you came to the conclusion.

Request example:
I want to cancel my subscription

Example response:
order_cancellation
"""

In [11]:
messages = [
    SystemMessage(
        content = prompt_orchestrator
    ),
    HumanMessage(
        #content="Can you give me information about the isolate protein?"
        content="Can you give me the status of my order?"
    ),
]

response = llm.invoke(messages)
print(response.content)

order_status


In [12]:
messages = [
    SystemMessage(
        content = prompt_orchestrator
    ),
    HumanMessage(
        content="Do you have Creatine Monohydrate stock?"
    ),
]

response = llm.invoke(messages)
print(response.content)

product_information


In [13]:
messages = [
    SystemMessage(
        content = prompt_orchestrator
    ),
    HumanMessage(
        content="Who is Leo Messi?"
    ),
]

response = llm.invoke(messages)
print(response.content)

NULL


## Respuesta al aviso

Una vez que ya tenemos categorizada la solicitud del usuario, ya podemos usar el aviso con el objetivo de obtener la mejor respuesta.


In [14]:
product_info_prompt = """
You are SuppleBot, an automated service to help customers place orders from a sports supplement website.
First, greet the customer warmly. Then, guide them through the inventory, offering product details
such as name, quantity available, and price. If a product is out of stock, the quantity will be marked as 0.
Collect the customer's order, verifying product selections and quantities.
After gathering the order, summarize it for the customer, and ask if they would like to add anything else.
For out-of-stock items, suggest alternative products.
Finally, proceed to collect payment information securely.
Ensure clear communication about product options, flavors, and sizes to uniquely
identify each item from the inventory.
Your responses should be short, conversational, and friendly.
If the request is away from your training just return NULL
The inventory includes
***
Whey Protein - 2 lbs, 25 in stock, $29.99
Vegan Protein - 1.5 lbs, 0 in stock, $27.99
Pre-Workout - 300g, 15 in stock, $39.99
BCAA - 400g, 20 in stock, $24.99
Creatine Monohydrate - 500g, 30 in stock, $19.99
Multivitamins - 60 capsules, 0 in stock, $12.99
Omega-3 Fish Oil - 120 capsules, 10 in stock, $15.99
Energy Bars - Box of 12, 50 in stock, $21.99
Hydration Tablets - 20 tablets, 40 in stock, $9.99
***
"""

In [15]:
messages = [
    SystemMessage(
        content = product_info_prompt
    ),
    HumanMessage(
        content="Do you have Creatine Monohydrate stock?"
    ),
]

response = llm.invoke(messages)
print(response.content)

Hi there! Yes, we have Creatine Monohydrate (500g) in stock—30 units available at $19.99 each. Would you like to add this to your order, or are you interested in any other supplements?


In [16]:
messages = [
    SystemMessage(
        content = product_info_prompt
    ),
    HumanMessage(
        content="I want to cancel my order..."
    ),
]

response = slm.invoke(messages)
print(response.content)

Hi there! I don’t see any order placed yet. How can I assist you with your supplement needs today?
